In [1]:
import numpy as np
from numpy import *
import matplotlib
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
import requests
import pickle
import re
import itertools
import functools
import collections
import string
import time
from bs4 import BeautifulSoup
from sortedcontainers import SortedList # http://www.grantjenks.com/docs/sortedcontainers/sortedlist.html#sortedlist

from adventlib import *

YEAR = 2022
DAY = int('09')

submit1, submit2 = generate_submits(YEAR, DAY)

while True:
  try:
    raw_input = get_input(YEAR, DAY)
    break
  except Exception as e:
    print(e)
    time.sleep(1)

In [2]:
lines, groups = linesplit("""
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
""".strip())

In [35]:
lines, groups = linesplit("""
R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20
""".strip())

In [62]:
lines, groups = linesplit(raw_input, verbose=True)

2000 lines
1 groups
>>>
R 1
U 1
D 1
...


In [63]:

DIR4 = (
  (0, 1),
  (1, 0),
  (0, -1),
  (-1, 0),
)

DIR8 = (
  (0, 1),
  (1, 1),
  (1, 0),
  (1, -1),
  (0, -1),
  (-1, -1),
  (-1, 0),
  (-1, 1),
)

DIRD = (
  (1, 1),
  (1, -1),
  (-1, -1),
  (-1, 1),
)

In [64]:
def touching(ax, ay, bx, by, dirs):
  for dx, dy in dirs:
    if ax + dx == bx and ay + dy == by:
      return True
  return False

In [24]:
ax, ay = 0, 0
bx, by = 0, 0

diffbs = set()
diffbs.add((bx, by))

for line in lines:
  if m := re.findall(r'^(.) (\d+)$', line):
    direc = m[0][0]
    steps = int(m[0][1])
    
    if direc == 'R':
      dx, dy = 1, 0
    if direc == 'U':
      dx, dy = 0, 1
    if direc == 'L':
      dx, dy = -1, 0
    if direc == 'D':
      dx, dy = 0, -1
      
    for i in range(steps):
      ax += dx
      ay += dy
      
      if touching(ax, ay, bx, by, DIR8) or (ax == bx and ay == by):
        print('already touching ', end='')
        print(ax, ay, bx, by)
        diffbs.add((bx, by))
        continue
      
      for ddx, ddy in DIR4:
        if touching(ax, ay, bx + ddx, by + ddy, DIR4):
          bx += ddx
          by += ddy
          print('move DIR4 ', end='')
          print(ax, ay, bx, by)
          diffbs.add((bx, by))
          break
      else:
        for ddx, ddy in DIRD:
          if touching(ax, ay, bx + ddx, by + ddy, DIR4):
            bx += ddx
            by += ddy
            
            print('MOVE DIRD ', end='')
            print(ax, ay, bx, by)
            diffbs.add((bx, by))
            break
    
  else:
    raise Exception()

already touching 1 0 0 0
already touching 1 1 0 0
already touching 1 0 0 0
already touching 1 1 0 0
MOVE DIRD 2 1 1 1
move DIR4 3 1 2 1
already touching 3 0 2 1
MOVE DIRD 3 -1 3 0
already touching 4 -1 3 0
MOVE DIRD 5 -1 4 -1
already touching 5 -2 4 -1
MOVE DIRD 5 -3 5 -2
already touching 6 -3 5 -2
already touching 6 -2 5 -2
already touching 5 -2 5 -2
already touching 5 -1 5 -2
move DIR4 5 0 5 -1
already touching 5 -1 5 -1
already touching 5 -2 5 -1
already touching 6 -2 5 -1
MOVE DIRD 6 -3 6 -2
already touching 7 -3 6 -2
MOVE DIRD 8 -3 7 -3
already touching 8 -2 7 -3
MOVE DIRD 8 -1 8 -2
already touching 9 -1 8 -2
MOVE DIRD 10 -1 9 -1
already touching 9 -1 9 -1
already touching 8 -1 9 -1
already touching 8 -2 9 -1
MOVE DIRD 8 -3 8 -2
already touching 7 -3 8 -2
MOVE DIRD 6 -3 7 -3
already touching 6 -2 7 -3
MOVE DIRD 6 -1 6 -2
already touching 6 -2 6 -2
already touching 6 -3 6 -2
already touching 5 -3 6 -2
MOVE DIRD 4 -3 5 -3
already touching 5 -3 5 -3
already touching 6 -3 5 -3
already

In [25]:
len(diffbs)

6384

In [26]:
submit1(_)

Submitting 6384...


True

In [65]:
parts = [[0, 0] for i in range(10)]

diffbs = set()
diffbs.add(tuple(parts[-1]))

for line in lines:
  if m := re.findall(r'^(.) (\d+)$', line):
    direc = m[0][0]
    steps = int(m[0][1])
    
    if direc == 'R':
      dx, dy = 1, 0
    if direc == 'U':
      dx, dy = 0, 1
    if direc == 'L':
      dx, dy = -1, 0
    if direc == 'D':
      dx, dy = 0, -1
      
    for i in range(steps):
      parts[0][0] += dx
      parts[0][1] += dy
      
      for j in range(9):
        print(parts)
        if touching(parts[j][0], parts[j][1], parts[j + 1][0], parts[j + 1][1], DIR8) or (parts[j][0] == parts[j+1][0] and parts[j][1] == parts[j+1][1]):
          continue
        else:
          for ddx, ddy in DIR4:
            if touching(parts[j][0], parts[j][1], parts[j+1][0] + ddx, parts[j+1][1] + ddy, DIR4):
              parts[j+1][0] += ddx
              parts[j+1][1] += ddy

              break
          else:
            for ddx, ddy in DIRD:
              if touching(parts[j][0], parts[j][1], parts[j+1][0] + ddx, parts[j+1][1] + ddy, DIR4):
                parts[j+1][0] += ddx
                parts[j+1][1] += ddy
                break
            else:
              for ddx, ddy in DIRD:
                if touching(parts[j][0], parts[j][1], parts[j+1][0] + ddx, parts[j+1][1] + ddy, DIRD):
                  parts[j+1][0] += ddx
                  parts[j+1][1] += ddy
                  break
              else:
                print('!!!', parts)
              
      diffbs.add(tuple(parts[-1]))
  else:
    raise Exception()

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [66]:
len(diffbs)

2734

In [67]:
submit2(_)

Submitting 2734...


True